In [1]:
# https://www.cnblogs.com/machangwei-8/p/15925358.html
from kubernetes import client, config
# 加载认证文件
# config.kube_config.load_kube_config("minikube_config.yml")
config.kube_config.load_kube_config("config.yml")

In [2]:
#获取CoreV1Api和BatchV1Api版本对象
core_v1 = client.CoreV1Api()  #用于操作核心v1 API组中的资源，如Pod、Service、Node、Namespace等
batch_v1 = client.BatchV1Api()  #用于操作Job等资源
apps_v1 = client.AppsV1Api()  #用于操作Deployment、DaemonSet、ReplicaSet和StatefulSet
custom_api = client.CustomObjectsApi()  #用于操作CRD

In [3]:
# 列出命名空间
for ns in core_v1.list_namespace().items:
    print(ns.metadata.name)

default
kube-node-lease
kube-public
kube-system
kuboard


In [4]:
# 列出所有nodes
for node in core_v1.list_node().items:
    name = node.metadata.name
    status = node.status.conditions[-1].type if node.status.conditions[-1].status == "True" else "NotReady"
    ip = node.status.addresses[0].address
    version = node.status.node_info.kubelet_version
    os = node.status.node_info.os_image
    print("name:{},status:{},ip:{},version:{},os:{}".format(name, status, ip,version,os))

name:bigdata1,status:Ready,ip:192.168.0.44,version:v1.17.4,os:CentOS Linux 7 (Core)
name:bigdata2,status:Ready,ip:192.168.0.18,version:v1.17.4,os:CentOS Linux 7 (Core)
name:bigdata3,status:Ready,ip:192.168.0.67,version:v1.17.4,os:CentOS Linux 7 (Core)
name:bigdata4,status:Ready,ip:192.168.0.95,version:v1.17.4,os:CentOS Linux 7 (Core)


In [5]:
# 列出所有service，对应命令：kubectl get service --all-namespaces -o wide
for service in core_v1.list_service_for_all_namespaces().items:
    print(service.metadata.name)
    print(service.spec.cluster_ip)
    print(service.metadata.namespace)
    print(service.spec.ports)
    print(service.spec.type)
    print(service.spec.selector)

kubernetes
10.96.0.1
default
[{'app_protocol': None,
 'name': 'https',
 'node_port': None,
 'port': 443,
 'protocol': 'TCP',
 'target_port': 6443}]
ClusterIP
None
nginx-deploy
10.108.228.121
default
[{'app_protocol': None,
 'name': 'kx8cc6',
 'node_port': None,
 'port': 80,
 'protocol': 'TCP',
 'target_port': 80}]
ClusterIP
{'k8s.kuboard.cn/name': 'nginx-deploy'}
kube-dns
10.96.0.10
kube-system
[{'app_protocol': None,
 'name': 'dns',
 'node_port': None,
 'port': 53,
 'protocol': 'UDP',
 'target_port': 53}, {'app_protocol': None,
 'name': 'dns-tcp',
 'node_port': None,
 'port': 53,
 'protocol': 'TCP',
 'target_port': 53}, {'app_protocol': None,
 'name': 'metrics',
 'node_port': None,
 'port': 9153,
 'protocol': 'TCP',
 'target_port': 9153}]
ClusterIP
{'k8s-app': 'kube-dns'}
kubelet
None
kube-system
[{'app_protocol': None,
 'name': 'https-metrics',
 'node_port': None,
 'port': 10250,
 'protocol': 'TCP',
 'target_port': 10250}, {'app_protocol': None,
 'name': 'http-metrics',
 'node_port':

In [6]:
# 列出指定命名空间的service
for service in core_v1.list_namespaced_service("default").items:
    print(service)

{'api_version': None,
 'kind': None,
 'metadata': {'annotations': None,
              'creation_timestamp': datetime.datetime(2023, 4, 11, 3, 9, 13, tzinfo=tzutc()),
              'deletion_grace_period_seconds': None,
              'deletion_timestamp': None,
              'finalizers': None,
              'generate_name': None,
              'generation': None,
              'labels': {'component': 'apiserver', 'provider': 'kubernetes'},
              'managed_fields': None,
              'name': 'kubernetes',
              'namespace': 'default',
              'owner_references': None,
              'resource_version': '148',
              'self_link': '/api/v1/namespaces/default/services/kubernetes',
              'uid': '3c631073-584a-4f8a-b24c-7e98b54c0b5a'},
 'spec': {'allocate_load_balancer_node_ports': None,
          'cluster_i_ps': None,
          'cluster_ip': '10.96.0.1',
          'external_i_ps': None,
          'external_name': None,
          'external_traffic_policy':

In [7]:
# 获取指定service信息
service = core_v1.read_namespaced_service(name="nginx-deploy", namespace="default")
service

{'api_version': 'v1',
 'kind': 'Service',
 'metadata': {'annotations': None,
              'creation_timestamp': datetime.datetime(2023, 6, 6, 2, 7, 16, tzinfo=tzutc()),
              'deletion_grace_period_seconds': None,
              'deletion_timestamp': None,
              'finalizers': None,
              'generate_name': None,
              'generation': None,
              'labels': {'k8s.kuboard.cn/name': 'nginx-deploy'},
              'managed_fields': None,
              'name': 'nginx-deploy',
              'namespace': 'default',
              'owner_references': None,
              'resource_version': '7033720',
              'self_link': '/api/v1/namespaces/default/services/nginx-deploy',
              'uid': '8ec78009-0e5b-40b9-a706-47e505e5438a'},
 'spec': {'allocate_load_balancer_node_ports': None,
          'cluster_i_ps': None,
          'cluster_ip': '10.108.228.121',
          'external_i_ps': None,
          'external_name': None,
          'external_traffic_poli

In [9]:
# 列出service绑定的pods信息
endpoints = core_v1.list_namespaced_endpoints(namespace="default", field_selector="metadata.name=nginx-deploy")
print(endpoints)
for ep in endpoints.items:
    print("IP: ", ep.subsets[0].addresses[0].ip)
    print("Port: ", ep.subsets[0].ports[0].port)

{'api_version': 'v1',
 'items': [{'api_version': None,
            'kind': None,
            'metadata': {'annotations': {'endpoints.kubernetes.io/last-change-trigger-time': '2023-06-06T02:07:16Z'},
                         'creation_timestamp': datetime.datetime(2023, 6, 6, 2, 7, 16, tzinfo=tzutc()),
                         'deletion_grace_period_seconds': None,
                         'deletion_timestamp': None,
                         'finalizers': None,
                         'generate_name': None,
                         'generation': None,
                         'labels': {'k8s.kuboard.cn/name': 'nginx-deploy'},
                         'managed_fields': None,
                         'name': 'nginx-deploy',
                         'namespace': 'default',
                         'owner_references': None,
                         'resource_version': '7033721',
                         'self_link': '/api/v1/namespaces/default/endpoints/nginx-deploy',
                     

In [15]:
# 列出指定ns的pod
for pod in core_v1.list_namespaced_pod("default").items:
    print("name:{},ip:{},ns:{}".format(pod.metadata.name ,pod.status.pod_ip,pod.metadata.namespace))

name:nginx-deploy-7669d4b87b-ft8cz,ip:10.244.103.85,ns:default
name:nginx-deploy-7669d4b87b-z5zl6,ip:10.244.34.150,ns:default


In [7]:
# 列出所有ns的pod
for pod in core_v1.list_pod_for_all_namespaces().items:
    print("name:{},ip:{},ns:{}".format(pod.metadata.name ,pod.status.pod_ip,pod.metadata.namespace))

name:nginx-deploy-7669d4b87b-ft8cz,ip:10.244.103.85,ns:default
name:nginx-deploy-7669d4b87b-z5zl6,ip:10.244.34.150,ns:default
name:calico-kube-controllers-6b94766748-f55lq,ip:10.244.191.220,ns:kube-system
name:calico-node-5sdft,ip:192.168.0.95,ns:kube-system
name:calico-node-cpq7b,ip:192.168.0.67,ns:kube-system
name:calico-node-tfj95,ip:192.168.0.44,ns:kube-system
name:calico-node-vqb2x,ip:192.168.0.18,ns:kube-system
name:coredns-6955765f44-f28m2,ip:10.244.191.222,ns:kube-system
name:coredns-6955765f44-wcfvm,ip:10.244.251.14,ns:kube-system
name:etcd-bigdata1,ip:192.168.0.44,ns:kube-system
name:kube-apiserver-bigdata1,ip:192.168.0.44,ns:kube-system
name:kube-controller-manager-bigdata1,ip:192.168.0.44,ns:kube-system
name:kube-proxy-29p4r,ip:192.168.0.44,ns:kube-system
name:kube-proxy-sch6n,ip:192.168.0.18,ns:kube-system
name:kube-proxy-w62dp,ip:192.168.0.67,ns:kube-system
name:kube-proxy-x8469,ip:192.168.0.95,ns:kube-system
name:kube-scheduler-bigdata1,ip:192.168.0.44,ns:kube-system
nam

In [8]:
# 查看crd资源
for cr in custom_api.list_namespaced_custom_object(group="stable.example.com",version="v1",namespace="default",plural="crontabs")["items"]:
    print(cr)